In [1]:
# Setup imports and path for local modules
import sys
sys.path.append("../src")

from data_preprocessing import load_and_preprocess
from hpo_optuna import run_optimization
from train_model import train
from batch_inference import batch_predict
# from drift_detection import detect_drift  # Optional bonus

import joblib
import mlflow
import pandas as pd

df = pd.read_csv("../data/diabetes.csv")
print(df.columns)
df.head()


# Step 1: Preprocess Data
X_train, X_test, y_train, y_test, scaler = load_and_preprocess("../data/diabetes.csv")
joblib.dump(scaler, "../models/scaler.pkl")  # Save scaler for inference




/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Index(['PatientID', 'Pregnancies', 'PlasmaGlucose', 'DiastolicBloodPressure',
       'TricepsThickness', 'SerumInsulin', 'BMI', 'DiabetesPedigree', 'Age',
       'Diabetic'],
      dtype='object')


['../models/scaler.pkl']

In [2]:
# Step 2: Hyperparameter Tuning with Optuna
best_params = run_optimization()
print("Best Parameters:", best_params)


[I 2025-07-16 18:34:17,419] A new study created in memory with name: no-name-78fab042-de23-4f77-b8dd-123483cafeb9
[I 2025-07-16 18:34:18,296] Trial 0 finished with value: 0.9722935555237378 and parameters: {'n_estimators': 114, 'max_depth': 2, 'learning_rate': 0.032076731441393926}. Best is trial 0 with value: 0.9722935555237378.
[I 2025-07-16 18:34:20,051] Trial 1 finished with value: 0.9846023989543257 and parameters: {'n_estimators': 61, 'max_depth': 8, 'learning_rate': 0.13703849148897584}. Best is trial 1 with value: 0.9846023989543257.
[I 2025-07-16 18:34:23,309] Trial 2 finished with value: 0.9897707275842798 and parameters: {'n_estimators': 158, 'max_depth': 6, 'learning_rate': 0.10802190749338356}. Best is trial 2 with value: 0.9897707275842798.
[I 2025-07-16 18:34:25,488] Trial 3 finished with value: 0.9904069539192004 and parameters: {'n_estimators': 127, 'max_depth': 5, 'learning_rate': 0.14751543409907036}. Best is trial 3 with value: 0.9904069539192004.
[I 2025-07-16 18:3

Best trial: {'n_estimators': 162, 'max_depth': 4, 'learning_rate': 0.16935699367965726}
Best Parameters: {'n_estimators': 162, 'max_depth': 4, 'learning_rate': 0.16935699367965726}


In [3]:
# Step 3: Train final model using best parameters
model, acc, auc = train(X_train, X_test, y_train, y_test, best_params)
print(f"Final Accuracy: {acc:.4f}, AUC: {auc:.4f}")





2025/07/16 18:35:19 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/16 18:35:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model saved to models/model.pkl
Final Accuracy: 0.9545, AUC: 0.9918
